In [1]:
import pandas as pd
import json

In [2]:
# Data load
with open('./data/train_original.json', 'rt', encoding='UTF8') as file :
    train_data = json.loads(file.read())

In [3]:
# json file architecture
# train_data

In [4]:
train_data_df = pd.DataFrame(train_data['documents'])

In [5]:
train_data_df.head(1)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...


In [6]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243983 entries, 0 to 243982
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       243983 non-null  object
 1   category                 243983 non-null  object
 2   media_type               243983 non-null  object
 3   media_sub_type           243983 non-null  object
 4   media_name               243983 non-null  object
 5   size                     243983 non-null  object
 6   char_count               243983 non-null  object
 7   publish_date             243983 non-null  object
 8   title                    243983 non-null  object
 9   text                     243983 non-null  object
 10  annotator_id             243983 non-null  int64 
 11  document_quality_scores  243983 non-null  object
 12  extractive               243983 non-null  object
 13  abstractive              243983 non-null  object
dtypes: int64(1), object(

In [18]:
train_data_df.category.value_counts()

종합           177558
경제            23938
사회            17650
정치            16389
스포츠            5174
IT,과학          1931
교육/입시/NIE      1190
부동산              74
보건/의료            55
기업               17
북한/한반도정세          5
선거                2
Name: category, dtype: int64

In [6]:
train_data_df.char_count = train_data_df.char_count.astype('int')

In [7]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass 
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [8]:
# test
art_sentence = extract_body(train_data_df['text'][2])
sentence_validation(art_sentence)

'전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. 임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를 달성하기 위한 결연한 의지를 다졌다. 이번 해맞이 행사는 2018년을 시작하면서 떠오르는 해를 보며 전남드래곤즈 구성원 모두가 한마음 한 뜻으로 구단 목표 달성을 위해 정진하자는 의미에서 실시한 것이다. 신승재 사장은“유상철 감독을 비롯한 코칭스텝, 선수단 구성이 마무리 된 만큼 구성원 모두가 하나되어 K리그 클래식 5위 이내 진입, FA컵 우승 등 ACL 진출권 획득을 목표로 최선을 다하자”고 선수들에게 신년 인사말을 전했다. 유상철 감독은“구봉산의 정기를 받아 2018년을 전남드래곤즈의 해로 만들겠다”며 각오를 다졌다. 한편 전남은 선수들도 추가 영입했다. 우선 프렌차이즈 스타 김영욱과 2020년까지 연장계약을 했다. 전남 유스 출신인 김영욱은 당초 계약기간이 1년 남아 있었지만, 전남의 프렌차이즈 스타로 팀의 중추적인 역할을 하고 있어 2020년까지 연장계약을 체결하게 됐다. 전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다. 완델손은 왼발 크로스가 정확하고 화려한 테크닉을 이용한 측면 공격력이 뛰어난 선수이다. 또한 윙 포워드는 물론 공격형 미드필더, 측면 수비수로도 뛸 수 있는 멀티 플레이어다. 공격력 강화를 위해 페체신 대체자로 중국 리그에서 활약한 하태균도 영입했다. 하태균은2007년 드래프트 1순위로 수원 삼성에 입단해 K리그 신인상을 수상한 공격수로 2014년까지 K리그 통산 135경기 30골 7도움을 기록했다. 2015년 중국으로 이적, 연변에서 26골을 기록 득점왕에 오르며 팀을 슈퍼리그로 승격시켰다. 또한 2017년에는 바오딩으로 이적해 팀 내 최다골인 14골을 기록할 정도로 득점력이 뛰어난 선수다. 이와 함께 박태준·이경렬·윤동민도 영입하며 올 시즌을 준비하고 있다.'

In [11]:
new_df = train_data_df.copy()
new_df.text = new_df.text.apply(lambda x : sentence_validation(extract_body(x)))
new_df.head(1).T

,0
id,290741778
category,종합
media_type,online
media_sub_type,지역지
media_name,광양신문
size,small
char_count,927
publish_date,2018-01-05 18:54:55
title,"논 타작물 재배, 2월 말까지 신청하세요"
text,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...


# Kobart 모델 파인튜닝

## 라이브러리 설치

In [10]:
!pip install datasets
!pip install transformers
!pip install load_metric
!pip install rouge_score
!pip install evaluate

  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached aiohttp-3.8.4-cp38-cp38-win_amd64.whl (324 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp38-cp38-win_amd64.whl (28 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-win_amd64.whl (34 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.8.2-cp38-cp38-win_amd64.whl (56 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 4.8.2 requires huggingface-hub==0.0.12, but you'll have huggingface-hub 0.13.4 which is incompatible.
requests 2.21.0 requires urllib3<1.25,>=1.21.1, but you'll have urllib3 1.26.15 which is incompatible.
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.4
    Uninstalling huggingface-hub-0.13.4:
      Successfully uninstalled huggingface-hub-0.13.4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

responses 0.18.0 requires urllib3>=1.25.10, but you'll have urllib3 1.24.3 which is incompatible.
datasets 2.11.0 requires huggingface-hub<1.0.0,>=0.11.0, but you'll have huggingface-hub 0.0.12 which is incompatible.
botocore 1.29.115 requires urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement load_metric (from versions: none)
ERROR: No matching distribution found for load_metric
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38

  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24992 sha256=8aa8630c5b783f90fe39114f26279648ae47518a8830fcd7f61b5dc81a901e8c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\24\55\6f\ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 4.8.2 requires huggingface-hub==0.0.12, but you'll have huggingface-hub 0.13.4 which is incompatible.
requests 2.21.0 requires urllib3<1.25,>=1.21.1, but you'll have urllib3 1.26.15 which is incompatible.
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


## 데이터 불러오기

In [11]:
from datasets import load_dataset,load_metric

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
raw_datasets = load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")
metric = load_metric("rouge")

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 199.99it/s]
C:\Users\User\AppData\Local\Temp\ipykernel_25020\1462125853.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


Dataset csv downloaded and prepared to C:/Users/User/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-884ccea06154613b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [13]:
raw_datasets

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 2740
})

In [14]:
raw_datasets = raw_datasets.train_test_split(test_size=0.2)

In [15]:
raw_datasets["test"][0]

{'date': '2022-07-06 15:11:02',
 'category': 'economy',
 'press': '헤럴드경제 ',
 'title': '환경부 녹색산업 혁신성장 옴부즈만 위촉',
 'document': '현장 목소리 청취 규제개선 과제 발굴 등 현장 소통창구 강화 헤럴드경제 김용훈 기자 환경부는 6일 서울 중구 프레지던트호텔에서 ‘녹색산업 혁신성장 옴부즈만 위촉식’을 열고 박정호 인선모터스 대표를 옴부즈만으로 연임한다고 밝혔다. 이에 따라 박정호 옴부즈만은 이날부터 2024년 7월 5일까지 2년간 ‘녹색산업 혁신성장 옴부즈만’으로서 녹색산업체의 어려움을 듣고 제도 개선방안 제안 등의 업무를 수행한다. 옴부즈만은 정부 행정에 대해 관련 공무원에게 설명을 요구하고 필요한 사항을 조사해 민원인에게 결과를 알려 주는 등의 활동을 하는 제도를 의미한다. 유제철 환경부 차관은 이날 박정호 옴부즈만에게 위촉장을 전달하면서 옴부즈만 직을 이어준 것에 대한 감사 인사와 함께 혁신에 걸림돌이 되는 규제개선 건의와 현장의 목소리 등을 가감 없이 전달해 주기를 요청할 예정이다. 박정호 옴부즈만은 “탄소중립 환경·사회·투명 경영 ESG 등 환경에 관한 관심에 부응하여 녹색산업이 성장할 수 있도록 현장 목소리를 잘 전달하도록 노력하겠다”고 연임 소감을 밝혔다. 환경부는 옴부즈만 제도 운용을 통해 새 정부의 규제개혁에 더욱 활력을 불어넣을 예정이다. 특히 기존 옴부즈만 운영상의 부족한 점을 보완하여 반기별로 회의를 정례화하고 미반영된 개선과제는 심층 검토회의를 통해 대안 마련 등 후속 조치를 시행할 계획이다. 한편 환경부는 옴부즈만 위촉식에 이어 환경산업계 7개 협·단체장과 환경기업 대표 등이 참여하는 간담회도 개최한다. 이날 유제철 차관과 박정호 옴부즈만은 탄소중립 실현 및 환경·사회·투명 경영 ESG 활성화를 위한 녹색산업계의 역할을 논의하고 이를 지원하기 위한 제도적 개선 방향을 함께 모색한다. 유제철 환경부 차관은 “녹색산업 혁신성장 옴부즈만이 정부와 녹색산업

## 데이터 확인

In [16]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [17]:
show_random_elements(raw_datasets["train"])

,date,category,press,title,document,link,summary
0,2022-07-04 10:07:02,economy,헤럴드경제,CJ대한통운 ‘블랙이글스’ 9대 운송…“최고난도 물류작전 성공”,英 국제 에어쇼 참가…분해·운송·재조립 과정 포함 8800㎞ 비행 이후 무진동 트레일러 실어 육상 이동 CJ대한통운이 영국 스탠스테드 공항에 도착한 블랙이글스 항공기 T 50B의 환적 작업을 진행하고 있다. CJ대한통운 제공 헤럴드경제 정찬수 기자 CJ대한통운이 영국 국제 에어쇼에 참가하는 공군 블랙이글스 Black Eagles T 50B 항공기 9대를 영국까지 운송하는 ‘블랙이글스 프로젝트’를 성공적으로 마쳤다고 4일 밝혔다. 프로젝트는 오는 9일부터 22일까지 영국에서 열리는 사우스포트·리아트·판보로 에어쇼에 참가하는 블랙이글스의 항공기와 장비 일체를 원주 공군기지부터 영국 보스콤다운 비행장까지 운송하는 것이다. 최첨단 정밀 부품과 고가 장비가 집약된 항공기 운송은 최고난도 물류로 알려져 있다. CJ대한통운은 성공적인 프로젝트 수행을 위해 지난 4월부터 준비했다. 화물전세기 3대 무진동 트레일러 27대 컨보이차량 18대 등 특수화물에 최적화한 장비와 전문인력을 투입했다. CJ대한통운은 안전하고 효율적인 운송을 위해 T 50B를 3대씩 나눠 진행했다. 같은 운송과정을 3번 반복해 총 9대의 항공기를 최종 목적지까지 전달했다. 먼저 동체 날개 수평꼬리날개 수직꼬리날개 엔진 등으로 분해된 T 50B는 특수 제작된 방수커버와 결박을 위한 탄성벨트로 포장해 원주에서 인천공항까지 육로로 운송했다. 이후 B747 400F 화물전세기에 실어 8800㎞를 이동한 뒤 영국 스탠스테드 공항에서 육상운송으로 전환하는 환적 작업을 진행했다. 길이 13m의 무진동 트레일러에 실린 항공기는 시속 60㎞ 안팎의 저속으로 운행하며 컨보이차량의 호위와 함께 보스콤다운 비행장까지 옮겨졌다. 최종 목적지에 도착한 T 50B는 재조립 과정을 거쳐 에어쇼에서 최상의 비행 퍼포먼스를 낼 수 있도록 준비된다. 에어쇼 이후에는 블랙이글스 조종사들이 직접 T 50B를 몰고 13개국을 경유하며 복귀한다. 폴란드 이집트 필리핀 등 여러 국가에서 에어쇼와 전시를 통해 방산수출에 기여한다는 계획이다. 장영호 CJ대한통운 포워딩본부장은 “크고 무거운 중량화물이나 취급하기 까다로운 특수화물운송 분야의 역량과 노하우를 바탕으로 고객이 원하는 어떠한 물류서비스도 수행할 수 있도록 노력하겠다”고 말했다. 한편 CJ대한통운은 지난 2012년에도 영국 에어쇼에 참가한 블랙이글스 T 50B 항공기 9기를 운송했다. 공군이 사용했던 T 38 훈련기 15대를 미 공군에 반환할 때도 육상 운송과 선적을 담당했다. 터키 고대유적 국립중앙박물관 유물 원전설비 대형극장 등 다양한 종류의 특수 화물운송을 수행한 노하우도 보유하고 있다. CJ대한통운이 영국 스탠스테드 공항에 도착한 블랙이글스 항공기 T 50B의 육상 운송을 시작하고 있다.,https://n.news.naver.com/mnews/article/016/0002010943?sid=101,CJ대한통운이 오는 9일부터 22일까지 영국에서 열리는 사우스포트·리아트·판보로 에어쇼에 참가하는 공군 블랙이글스 Black Eagles T 50B 항공기 9대를 오는 9일부터 22일까지 원주 공군기지부터 영국 보스콤다운 비행장까지 운송하는 '블랙이글스 프로젝트'를 성공적으로 마쳤다고 4일 밝혔다.
1,2022-07-01 12:10:52,economy,뉴스1,김치 등 단순가공식품 부가세 면제,세종 뉴스1 김기남 기자 민생안정 대책의 일환으로 병·캔 등으로 개별 포장된 김치 된장 고추장 간장 젓갈류 등 단순가공식품의 부가가치세 10% 인하 첫날인 1일 오전 세종시 이마트 세종점에서 직원이 상품을 정리하고 있다. 정부는 2023년까지 단순 가공 식료품의 부가가치세 10%를 면제한다고 최근 발표했다.,https://n.news.naver.com/mnews/article/421/0006191613?sid=101,"정부가부는 2023년까지 단순 가공 식료품의 부가가치세 10%를 면제하고 2023년까지 2023년까지 단순 가공 식료품의 부가가치세 10%를 면제한다고 최근 발표했으나, 민생안정 대책의 일환으로 병·캔 등으로 개별 포장된 김치 된장 고추장 간장 젓갈류 등 단순가공식품의 부가가치세 10% 인하 첫날인 1일 오전 세종시 이마트 세종점에서 직원이 상품을 정리하고 있다."
2,2022-07-05 17:04:01,IT과학,조선비즈,‘수학계 노벨상’ 필즈상은 무엇... 40세 미만만 수상 가능,허준이 39 미국 프린스턴대 교수가 한국인 최초로 ‘수학계 노벨상’으로 불리는 필즈상 Fileds Medal 을 수상했다. 국제수학연맹 IMU 은 5일 “필즈상 수상자로 허 교수를 비롯해 마리나 비아조우스카 38 스위스 로잔연방공과대 교수 위고 뒤미닐코팽 37 프랑스 고등과학원 교수 제임스 메이나드 35 영국 옥스퍼드대 교수를 선정했다”고 밝혔다. 1936년부터 세계 수학계에서 탁월한 공적을 인정받은 40세 미만 젊은 학자에게 4년에 한 번씩 수여되는 필즈상은 ‘수학의 노벨상’이라 불릴 정도로 권위가 높다. 아무리 좋은 연구 결과가 있다 하더라도 나이 제한에 따라 40세가 되면 상을 받지 못하는 것이다. 이 상이 제정되기까지는 캐나다 출신 수학자 존 찰스 필즈의 공로가 있었다. 필즈는 1863년 5월14일 캐나다 온타리오주 해밀턴에서 가죽 상인의 아들로 태어났다. 토론토대 수학과를 졸업한 뒤 미국 존스홉킨스대에서 수학 박사학위를 땄고 유럽으로 건너가 연구원 생활을 했다. 필즈상 메달의 모습 과기정통부 필즈는 1902년 토론토대 수학과 교수가 된 뒤 온타리오주 의회를 설득해 수학 연구에 매년 7만5000달러를 지원한다는 약속을 받아냈다. 또 1924년 국제수학자대회 ICM 를 토론토에 유치했다. 이 행사에서 “수학자만을 위한 상을 제정해야 한다”고 처음 주창했다. 이후 유언으로 재산 4만7000달러를 기금으로 내놓기도 했다. 하지만 1932년 8월9일 69세에 심장마비로 갑자기 별세하면서 생전엔 상이 주어지는 걸 보지 못했다. 그래도 평생 수학을 사랑한 그의 이름은 상의 명칭으로 남았다. 필즈의 노력으로 1936년 첫 수상자가 나왔다. 핀란드의 라르스 발레리안 알포르스는 ‘유리형 함수 및 리만 면의 연구’라는 주제로 필즈상을 최초로 받았다. 미국의 제시 더글러스도 ‘플라토의 문제’라는 주제로 필즈상을 받았다. 필즈상은 1966년까지 매번 2명의 수상자를 선정했다. 하지만 시간이 지나 수학 분야가 점점 넓어지면서 뛰어난 업적이 많이 나왔고 1966년부터 수상자를 4명으로 늘렸다. 현재까지 총 60명이 필즈상을 수상했다. 아시아에서는 일본 3명 중국 1명 등 총 8명의 수상자가 나왔다. 주요 수상자로는 1954년 프랑스 수학자의 장 피에르 세르는 스펙트럼 수열의 방법을 통한 구면의 호모토피 군에 관한 계산 및 층의 이론을 사용하여 다변수복소해석학을 재정립·확장해 필즈상을 받았다. 그의 나이 27세의 최연소 수상 기록이다. 2014년에는 이란의 수학자 마리암 미르자카니가 여성 최초로 필즈상을 받았다. 그는 모듈라이 공간의 부피를 측정하는 성과를 인정받았다. 필즈상 수상자에게 수여하는 메달의 공식 이름은 ‘수학에서 뛰어난 발견에 관한 국제 메달’이다. 메달 앞면에 고대 그리스 수학자 아르키메데스의 초상과 함께 라틴어로 ‘자신 위로 올라서 세상을 꽉 붙잡아라’ 하는 문구가 적혀 있다. 뒷면에는 ‘전 세계에서 모인 수학자들이 탁월한 업적에 상을 수여한다’고 적혀 있다. 존 필즈는 이 상에 대해 ‘이미 완성된 업적을 표창하지만 이 상을 받은 사람은 그 분야에서 더 뛰어난 성취를 위해 용기를 북돋우며 다른 새로운 분야의 노력을 자극한다는 것을 알게 될 것이다’라는 메모를 남겼다. 이 메모를 당시 수학자들은 젊은 수학자에게 주는 상이라고 해석했다. 이에 필즈상은 수상자를 40세 미만 젊은 수학자로 제한하고 있다. 정확하게 상을 주는 해의 1월 1일을 기준으로 40세가 되지 않은 수학자가 대상인 것이다. 페르마의 마지막 정리를 증명한 앤드루 와일스는 논문을 41세에 완성해 필즈상을 

In [18]:
raw_datasets["test"]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 548
})

In [19]:
raw_datasets["train"]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 2192
})

# 데이터 전처리 함수

In [8]:
#kobart 불러오기
from transformers import PreTrainedTokenizerFast, BartModel

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
model = BartModel.from_pretrained('gogamza/kobart-base-v2')

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [21]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[16391, 16997, 20858, 243, 19133, 15868, 1700, 14889, 300, 18482, 16884, 15265, 22038, 232], [15054, 303, 15868, 1700, 15868, 1700, 15195, 310, 16160, 14879, 18482, 16884, 15265, 22038, 245]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
max_input_length = 1024
max_target_length = 128
# prefix = "summarize: "
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
# 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [23]:
raw_datasets['train'][0]["document"]

'앵커 안 오른 게 없지만 커피 한 잔 가격도 많이 올랐죠. 정부가 최근에 커피 생원두를 수입할 때 붙었던 부가가치세를 면제하기 시작했는데요. 그럼 커피값이 내려가는 걸까요. 안보겸 기자가 알아봤습니다. 기자 더운 여름 더 즐겨 찾는 대형 프랜차이즈 커피 전문점. 맛이 정형화돼 있는 프랜차이즈 카페를 많이 애용하는 편이에요. 프랜차이즈가 훨씬 더 대로변에 많이 있는 경우가 많고 부쩍 오른 커피값에 정부는 지난달 28일부터 수입한 커피 생원두의 부가가치세 10%를 면제하고 있지만 프랜차이즈 커피값은 정작 내려갈 조짐이 보이지 않습니다. 스타벅스와 커피빈은 볶은 원두를 수입해 면세 혜택이 적용되지 않고 투썸플레이스 이디야 등은 생원두를 수입하지만 직접 로스팅해 부가세를 환급받고 있기 때문입니다. 커피 생두 수입업체들은 줄어든 세금 만큼 유통 가격을 내린다고 밝힌 상황. 정부는 커피 생두 면세 조치 효과가 8월부터 나타날 것이라고 내다봤지만 일반 카페 업주들의 생각은 달랐습니다. 고장수 전국 카페사장협동조합 이사장 우유 가격이 상승을 많이 했고요. 컵이랑 뚜껑의 가격이 계속 인상이 되고 있는 상황이기 때문에 인하된다 이런 일은 없을 것 같아요. 소규모 카페들 역시 지난 1년간 생두 가격이 60% 넘게 오르는 등 재료비가 올라 가격을 내릴 여력이 없다는 겁니다. 아메리카노 한 잔의 원두값은 500원 정도인데 부가세 10%가 빠져봐야 50원 줄어들 뿐입니다. 소비자들은 가격 부담을 호소합니다. 박예빈 인천 미추홀구 커피값이 점점 오르고 있는 거 같아서 사실 자주 가긴 하는데 부담스럽긴 하죠. 이번 대책이 커피값 추가 인상을 막을 뿐 인하 효과는 없을 것이란 지적이 나옵니다. 채널A 뉴스 안보겸입니다.'

In [24]:
raw_datasets['train'][0]["summary"]

'커피 급격히 올라 오른 커피값에 정부는 지난달 28일부터 커피 생원두의 부가가치세 10%를 면제하였지만 커피 생두 수입업체들은 줄어든 세금 만큼 유통 가격을 내린다고 하여 커피 생두 커피값은 정작 내려갈 조짐이 보이지 않는다.'

In [25]:
max_input_length = 1024
max_target_length = 128
# prefix = "summarize: "
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
# 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [26]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[26001, 14105, 15830, 14262, 21280, 17646, 14036, 15580, 14899, 9866, 14605, 19360, 17984, 15530, 26079, 17646, 14106, 11936, 24283, 15843, 13594, 14089, 15253, 15912, 26954, 12863, 14896, 22566, 14191, 14351, 17718, 14543, 16875, 17646, 22702, 15199, 14542, 14366, 9286, 14543, 17877, 9081, 28655, 15706, 10910, 15170, 14680, 14166, 11914, 16256, 14166, 23467, 17379, 15582, 24988, 17646, 14582, 12136, 245, 24244, 28320, 13714, 9895, 14082, 24988, 18654, 10443, 14605, 14496, 11908, 14049, 14433, 15958, 14543, 24988, 8981, 16719, 14166, 17397, 10869, 11786, 14605, 14082, 16605, 23133, 14062, 12423, 15830, 17646, 8997, 11786, 15804, 15260, 15450, 15243, 15843, 13590, 17646, 14106, 11936, 9925, 12024, 26954, 12863, 11285, 14239, 16445, 22566, 14058, 15651, 24988, 17646, 8997, 12005, 21094, 15199, 8989, 25185, 12034, 17727, 14076, 15170, 15123, 10831, 17127, 17646, 26286, 1700, 10886, 12005, 14145, 24283, 15843, 13607, 23688, 27663, 15101, 15051, 14672, 14233, 11538, 17117, 11

In [27]:
tokenized_data = raw_datasets.map(preprocess_function, batched=True)

In [28]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v2')


In [21]:
batch_size = 4
model_name = 'awesome_model'
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=False,
)

NameError: name 'Seq2SeqTrainingArguments' is not defined

In [32]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [45]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: name 'args' is not defined

In [36]:
# trainer.train()

In [28]:
tokenizer.bos_token_id

1

In [29]:
text = """ 
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""


In [30]:
text = text.replace('\n', ' ')


In [31]:
from transformers import pipeline


In [32]:
trainer.save_model()

In [33]:
summarizer = pipeline("summarization", model="awesome_model-finetuned-xsum", min_length=128, max_length=128, tokenizer='awesome_model-finetuned-xsum')
summarizer2 = pipeline("summarization", model="gogamza/kobart-base-v2", min_length=128, max_length=128, tokenizer='gogamza/kobart-base-v2')



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [34]:
summarizer(text)

[{'summary_text': '1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진되어 역대 최다 기록인 역대 최다 기록인 누적 확진자 수가 20만명대를 넘어선 것은 이번이 처음으로, 사상 처음 20만명대에 진입했다.  1일 오후 9시까지 최소 20만3220명이 신규 확진돼 역대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다. 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985'}]

In [35]:
summarizer2(text)

[{'summary_text': ' 1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다. 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다. 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음 처음이다. 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.'}]

# AI-hub data에 적용

In [12]:
# 추가 전처리 
art_df = new_df.copy()
art_df.drop(['id', 'annotator_id', 'char_count', 'document_quality_scores', 'extractive'], axis=1, inplace=True)

In [13]:
art_df.abstractive = art_df.abstractive.apply(lambda x : ' '.join(x))

In [14]:
pd.options.display.max_colwidth = 2000
art_df.head(1)

,category,media_type,media_sub_type,media_name,size,publish_date,title,text,abstractive
0,종합,online,지역지,광양신문,small,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .


In [15]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(art_df, test_size=0.2, random_state=777)

In [16]:
train_df['text'].str.len().sort_values(ascending=False)[:5] # 내림차순

66068     13910
36893      1963
182051     1959
187153     1955
105903     1954
Name: text, dtype: int64

In [17]:
# 위 결과 문자열 길이 1위와 2위의 차이가 큰 관계로 1위 데이터는 삭제
train_df.drop([66068], axis=0, inplace=True)

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
train_df['abstractive'].str.len().sort_values(ascending=False)[:5] # 내림차순

149185    716
152576    638
170525    610
186499    607
5598      498
Name: abstractive, dtype: int64

In [19]:
max_input_length = 2000
max_target_length = 750
prefix = ""

def preprocess_function(feature_list, target_list):
    inputs = [prefix + doc for doc in feature_list]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
# 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_list, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
preprocess_function(train_df.text[:2].values.tolist(), train_df.abstractive[:2].values.tolist())

{'input_ids': [[14025, 9496, 11806, 28090, 14493, 14141, 16831, 26577, 21988, 14140, 20701, 14059, 14040, 29711, 14680, 14180, 18145, 15164, 14058, 15964, 14025, 9496, 11806, 28090, 14493, 16838, 10338, 25831, 15086, 28191, 14363, 29388, 14305, 14228, 11268, 12147, 15389, 14412, 14099, 12074, 11372, 23226, 15097, 14130, 14099, 12074, 11372, 14281, 15640, 20678, 14049, 14456, 14025, 14281, 14321, 14295, 15295, 27816, 26690, 14030, 15166, 14081, 13737, 14382, 14301, 10884, 18332, 14519, 11319, 8995, 14036, 10504, 11667, 17983, 22262, 14130, 14948, 11763, 18373, 27386, 14687, 14864, 17855, 22893, 14081, 16059, 10219, 14205, 17596, 14543, 14376, 28924, 15253, 19248, 14205, 14209, 14120, 14142, 11786, 20914, 20196, 14776, 14543, 14948, 9495, 10449, 14320, 15060, 12332, 14672, 28484, 14117, 17930, 14130, 14381, 16579, 16005, 14972, 14334, 9908, 14910, 17657, 27417, 14910, 23609, 14076, 14130, 14948, 21698, 11224, 19545, 14429, 14082, 21934, 17657, 17942, 25382, 14749, 11911, 11467, 14142, 14